#Instalando bibliotecas/pacotes

In [ ]:
#Instalando
!pip install rasterio spectral -q

# Leitura do arquivo

In [ ]:
# Importando Bibliotecas
import numpy as np
import rasterio as rio
from spectral import imshow
import matplotlib.pyplot as plt
from rasterio.plot import reshape_as_image
from osgeo import gdal
import pandas as pd

In [ ]:
# Leitura do arquivo de imagem
img = '/content/L71221071_07120010720_DN.tif'

In [ ]:
# Rasterio
with rio.open(img) as src:
  meta = src.profile
  array_rasterio = reshape_as_image(src.read())

In [ ]:
print(meta)

# Visualização

In [ ]:
# Visualização
imshow(array_rasterio,(2,1,0), stretch=(0.02,0.98))

In [ ]:
def stretch(img, percent_ini=2, percent_end=98):

  s = np.zeros_like(img)
  x,y = np.min(img), np.max(img)
  w = np.percentile(img, percent_ini)
  z = np.percentile(img, percent_end)

  f = x + (img - w) * (y - x) / (z - w)

  f[f < x] = x
  f[f > y] = y
  s = f

  return s


In [ ]:
# Visualizando bandas separadas
fig, axes = plt.subplots(2,3, figsize=(10,8))

axes = axes.ravel()

for i in range(array_rasterio.shape[2]):
  axes[i].imshow(stretch(array_rasterio[:,:,i]), cmap = 'gray')
  axes[i].set_title('Banda' + str(i+1), fontsize=10)
  axes[i].axis('off')
plt.show()

In [ ]:
# Visualizando histogramas

colors = ['Blue', 'Green', 'Red', 'Maroon', 'Purple', 'Pink']
fig, axes = plt.subplots(2,3, figsize=(15,8))

axes = axes.ravel()

for i in range(array_rasterio.shape[2]):
  axes[i].hist(array_rasterio[:,:,i].flatten(), bins=50, color= colors[i])
  axes[i].set_title('Histograma da Banda' + str(i+1), fontsize=10)
plt.show()

In [ ]:
array_rasterio.reshape(array_rasterio.shape[0]*array_rasterio.shape[1], array_rasterio.shape[2]).shape

In [ ]:
# Criando pandas Dataframe
df = pd.DataFrame(array_rasterio.reshape(array_rasterio.shape[0]*array_rasterio.shape[1], array_rasterio.shape[2]),
columns = ['B1', 'B2', 'B3','B4', 'B5', 'B6'])
df.head()

In [ ]:
# Matriz de correlação
corr = df.corr()

fig = plt.figure(figsize=(20,20))

plt.matshow(corr)
plt.colorbar()
plt.title('Matriz de correlação')
plt.xticks(range(len(corr.columns)), corr.columns, rotation = 45)
plt.yticks(range(len(corr.columns)), corr.columns, rotation = 45)

# Colocando rótulos na matriz
for i in range(len(corr.columns)):
    for j in range(len(corr.columns)):
        text = plt.text(j, i, round(corr.iloc[i, j],2),
                       ha="center", va="center", color="Black")

plt.show()

In [ ]:
# Matriz de covariância
cov = df.cov()

fig = plt.figure(figsize=(20,20))

plt.matshow(cov)
plt.colorbar()
plt.title('Matriz de covariância')
plt.xticks(range(len(cov.columns)), cov.columns, rotation = 45)
plt.yticks(range(len(cov.columns)), cov.columns, rotation = 45)

# Colocando rótulos na matriz
for i in range(len(cov.columns)):
    for j in range(len(cov.columns)):
        text = plt.text(j, i, round(cov.iloc[i, j],2),
                       ha="center", va="center", color="Black")

plt.show()

# Operações raster

In [ ]:
# índices espectrais

NDVI = (array_rasterio[...,3] - array_rasterio[...,2]) / (array_rasterio[...,3] + array_rasterio[...,2])
NDWI = (array_rasterio[...,1] - array_rasterio[...,3]) / (array_rasterio[...,1] + array_rasterio[...,3])

In [ ]:
# visualização

fig, axes = plt.subplots(1,2, figsize=(15,10))
axes = axes.ravel()

# NDVI
im0 = axes[0].imshow(stretch(NDVI), cmap = 'RdYlGn')
axes[0].set_title('NDVI', fontsize=10)
axes[0].axis('off')
fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

# NDWI
im1 = axes[1].imshow(stretch(NDWI), cmap = 'Blues')
axes[1].set_title('NDWI', fontsize=10)
axes[1].axis('off')
fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)


In [ ]:
# Histograma do NDVI
plt.hist(np.clip(NDVI.flatten(), -1,1), bins=50)
plt.title('Histograma do NDVI')
plt.show()

# Classificação não-supervisionada


In [ ]:
# Reformatando dimensões
data = array_rasterio.reshape(array_rasterio.shape[0]*array_rasterio.shape[1], array_rasterio.shape[2])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Método do cotovelo
wcss = []
for i in range(1,11):
  k_means = KMeans(n_clusters=i, max_iter = 30, random_state = 10)
  k_means.fit(data)
  wcss.append(k_means.inertia_)

plt.plot(range(1,11), wcss)
plt.title('Método do cotovelo')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Rodando o kmeans
k_means = KMeans(n_clusters=6, max_iter = 50, random_state = 10)
k_means.fit(data)

labels = k_means.labels_
pred = labels.reshape(array_rasterio.shape[0], array_rasterio.shape[1])

In [ ]:
# Visualizar resultado
plt.imshow(pred, cmap = 'Spectral_r')
plt.show()

In [ ]:
# Gravar resultado em disco
meta.update(count=1)
with rio.open('kmeans.tif', 'w', **meta) as dst:
  dst.write(pred,1)